In [2]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle


In [98]:
def pipeline_model(path):
    # Face detection model
    faceDetectionModel =  'C:/Users/ASUS/Project/Data/res10_300x300_ssd_iter_140000_fp16.caffemodel'
    # Mô tả kiến trúc
    faceDetectionProto = 'C:/Users/ASUS/Project/Data/deploy.prototxt'
    # Mô hình mô tả khuôn mặt
    faceDescriptorModel = 'C:/Users/ASUS/Project/Data/openface.nn4.small2.v1.t7'
    # Sử dụng OpenCV DNN đọc mô hìn nhận diện khuôn mặt
    detectorModel = cv2.dnn.readNetFromCaffe(faceDetectionProto, faceDetectionModel)
    # Đọc mô hình mô tả khuôn mặt từ file Torch
    descriptorModel = cv2.dnn.readNetFromTorch(faceDescriptorModel)
    # face recognition
    face_recognition_model = pickle.load(open('./Pillow_01/img/ml_face_person_identity.pkl',mode = 'rb'))
    # pipeline model
    img1 = cv2.imread(path)
    img = img1.copy()
    h ,w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(img,1,(300,300),(123,174,106),swapRB = False,crop = False)
    detectorModel.setInput(blob)
    detection = detectorModel.forward()
    # machcine results
    machinlearning_results = dict(face_detect_score = [],
                                 face_name = [],
                                 face_name_score = [],
                                 count = [])
    count = 1
    if len(detection) > 0:
        for i,confidence in enumerate(detection[0,0,:,2]):
            confidence = detection[0,0,i,2]
            if confidence > 0.5:
                box = detection[0,0,i,3:7] * np.array([w,h,w,h])
                (start_x,start_y,end_x,end_y) = box.astype('int')
                cv2.rectangle(img,(start_x,start_y),(end_x,end_y),(0,255,0))
            # feature extraction
            roi = img[start_y:end_y,start_x:end_x]
            roiblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB=True,crop=True)
            descriptorModel.setInput(roiblob)
            vectors = descriptorModel.forward()
            # predict with machine learning
            face_name = face_recognition_model.predict(vectors)[0]
            face_score = face_recognition_model.predict_proba(vectors).max()
            text_face = f'{face_name}: {face_score *100:.0f}%'
            cv2.putText(img,text_face,(100,100),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            machinlearning_results['count'].append(count)
            machinlearning_results['face_detect_score'].append(confidence)
            machinlearning_results['face_name'].append(face_name)
            machinlearning_results['face_name_score'].append(face_score)
            count += 1
    return img , machinlearning_results

In [100]:
img, results = pipeline_model('./Pillow_01/Faces/Messi/messi.jpg')
#cv2.imshow('Image',img)

In [60]:
def display(img):
    cv2.imshow('Image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [102]:
display(img)